## FP-trees: an efficient way to encode a dataset

The FP-growth algorithm stores data in a compact data structure called an  FP-tree.(The FP stands for "frequent pattern.")

Sample transaction dataset |  An example FP-tree
:-------------------------:|:-------------------------:
![](./figs/table_12.1.jpg) |![](./figs/12-1.jpg)

- A path will tell you how many times a sequence occurred.
- A node identifies a single item from the set and the number of times it occurred in this sequence

### Creating the FP-tree data structure

In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name  = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numoccur):
        self.count += numoccur
    
    def disp(self, ind=1):
        print(" "*ind, self.name,' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
rootNode = treeNode('pyramid', 9 ,None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


In [3]:
rootNode.children['phoenix'] = treeNode('phoenix',3,None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


### Constructing the FP-tree

In [4]:
def createTree(dataSet, minSup=1):
    # to quickly access all of the elements of a given type in the FP-tree
    headerTable = {}
    # count the frequency of occurrence of each item.
def createTree(dataSet, minSup=1): 
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # eliminate any items that don't meet the minimum support
    for k in list(headerTable.keys()):               
        if headerTable[k] < minSup:                  
            del(headerTable[k])
    # get the set
    freqItemSet = set(headerTable.keys())
    
    if len(freqItemSet) == 0: return None, None
    
    for k in headerTable:
        # hold a count and pointer to the first item
        headerTable[k] = [headerTable[k], None]
    # building the tree started from null set
    retTree = treeNode('Null Set', 1, None)
    
    for tranSet, count in dataSet.items():
        localD = {}

        for item in tranSet:                        
            if item in freqItemSet:                      
                localD[item] = headerTable[item][0] 
                
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), 
                                     key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree,         
                       headerTable, count)            
    return retTree, headerTable 

In [5]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count) 
    else:   
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],
                                inTree.children[items[0]])
    if len(items) > 1:   
        updateTree(items[1::], inTree.children[items[0]], 
                                   headerTable, count)   

In [6]:
def updateHeader(nodeToTest, targetNode): 
    while (nodeToTest.nodeLink != None): 
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [7]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [8]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [9]:
simDat = loadSimpDat()
simDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [10]:
initSet = createInitSet(simDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [11]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     t   3
      s   2
       y   2
      r   1
       y   1
   x   1
    s   1
     r   1


In [12]:
myHeaderTab

{'z': [5, <__main__.treeNode at 0x1fb3955d978>],
 'r': [3, <__main__.treeNode at 0x1fb3955dac8>],
 't': [3, <__main__.treeNode at 0x1fb3955dfd0>],
 'x': [4, <__main__.treeNode at 0x1fb3955d7f0>],
 's': [3, <__main__.treeNode at 0x1fb3955d780>],
 'y': [3, <__main__.treeNode at 0x1fb3955df28>]}

## Mining frequent items from an FP-tree
### Extracting conditional pattern bases

In [13]:
def ascendTree(leafNode, prefixPath): 
    if leafNode.parent != None:                   
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [14]:
def findPrefixPath(basePat, treeNode): 
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [15]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [16]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [17]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'t', 'x', 'z'}): 1}

### Creating conditional FP-trees

In [18]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
    for basePat in bigL:                                 
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)                             
        if myHead != None:     
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [19]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'t'}
  Null Set   1
   z   3
    x   3
conditional tree for:  {'x', 't'}
  Null Set   1
   z   3
conditional tree for:  {'s'}
  Null Set   1
   x   3
conditional tree for:  {'y'}
  Null Set   1
   z   3
    t   2
     x   2
    x   1
     t   1
conditional tree for:  {'y', 't'}
  Null Set   1
   z   3
conditional tree for:  {'y', 'x'}
  Null Set   1
   z   3
conditional tree for:  {'x'}
  Null Set   1
   z   3


In [20]:
freqItems

[{'r'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'y', 'z'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## Example: finding co-occurring words in a Twitter feed